In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Add, AveragePooling2D, Concatenate
from tensorflow.keras.models import Sequential
from keras.layers import concatenate
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Load dataframe

In [ ]:
import os
Directory = '/Users/khunnoot/Desktop/year4/senior_project/jupyter_notebook'
CNNsDataframe = 'Image_dataset_1layer.csv'
AtributeDataframe = 'Attribute_dataset_1layer.csv'

In [ ]:
df = pd.read_csv(os.path.join(Directory, CNNsDataframe),parse_dates=True,index_col='datetime')
ATT = pd.read_csv(os.path.join(Directory, AtributeDataframe),parse_dates=True,index_col='datetime')

# Split data

In [ ]:
#define split data by using clear-sky index condition

def train_test_splits(df, test_size=0.2, random_state=42, split_by_k = True):
        df['date'] = df.index.date
        INDEX = df[['site','k']].copy()
        INDEX['date']= INDEX.index.date
        INDEX = INDEX.groupby(by=[df.index.date,'site']).mean()
        INDEX.reset_index(level=1, inplace=True)
        
        if split_by_k :
            klow = INDEX[INDEX.k<=0.3]
            kmed = INDEX[(INDEX.k>0.3) & (INDEX.k<=0.6)]
            khigh =  INDEX[(INDEX.k>0.6)]
            klow_test = klow.sample(frac=test_size,random_state=random_state)
            kmed_test = kmed.sample(frac=test_size,random_state=random_state)
            khigh_test = khigh.sample(frac=test_size,random_state=random_state)
            X_test = pd.concat([klow_test, kmed_test, khigh_test])

        else:
            X_test = INDEX.sample(frac=test_size, random_state = random_state)
            
        X_test.reset_index(inplace=True)
        X_test = X_test.rename(columns={'index':'date'})
        X_test = X_test.drop(columns = ['k'])
        X_test = df.reset_index().merge(X_test,on=['date','site']).set_index('datetime')
        X_train = pd.concat([df,X_test]).drop_duplicates(keep=False)
        X_train.drop(columns=['date'],inplace = True); X_test.drop(columns=['date'],inplace = True)
        
        return X_train, X_test

In [ ]:
#train test split
train_data, test_data = train_test_splits(df, test_size=0.2, random_state=42, split_by_k = True)
train_ATT, test_ATT = train_test_splits(ATT, test_size=0.2, random_state=42, split_by_k = True)

# Cloud2K Model

In [ ]:
Cloud2K_name = ['Cloud2Kv1','Cloud2Kv2','Cloud2Kv3','Cloud2Kv4']

In [ ]:
def resblock(x, kernelsize, filters):
    fx = layers.Conv2D(filters, kernelsize, padding='same')(x)
    fx = layers.BatchNormalization()(fx)
    act1 = layers.Activation(activation='relu')(fx)
    fx = layers.Conv2D(filters, kernelsize, padding='same')(act1)
    out = layers.Add()([x,fx])
    out = layers.BatchNormalization()(out)
    out = layers.ReLU()(out)
    return out

In [ ]:
Cloud2K_list = []
# Define the CNN architecture
input_shape = (16, 16, 1)
img_inputs = keras.Input(shape=input_shape)
conv1 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(img_inputs)
conv2 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv2)
flatten_cnn = layers.Flatten()(pool1)
dense1 = layers.Dense(16, activation='relu')(flatten_cnn)
dense2 = layers.Dense(8, activation='relu')(dense1)
outputs = layers.Dense(1)(dense2)
Cloud2Kv1 = keras.Model(inputs=img_inputs, outputs=outputs, name="Cloud2Kv1")
Cloud2K_list.append(Cloud2Kv1)
Cloud2Kv1.summary()

# Define the CNN architecture
input_shape = (16, 16, 1)
img_inputs = keras.Input(shape=input_shape)
conv1 = layers.Conv2D(32, kernel_size=(5, 5), activation='relu',padding='same')(img_inputs)
conv2 = layers.Conv2D(32, kernel_size=(5, 5), activation='relu',padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv2)
flatten_cnn = layers.Flatten()(pool1)
dense1 = layers.Dense(16, activation='relu')(flatten_cnn)
dense3 = layers.Dense(8, activation='relu')(dense1)
outputs = layers.Dense(1)(dense3)
Cloud2Kv2 = keras.Model(inputs=img_inputs, outputs=outputs, name="Cloud2Kv2")
Cloud2K_list.append(Cloud2Kv2)
Cloud2Kv2.summary()

# Define the CNN architecture
input_shape = (16, 16, 1)
img_inputs = keras.Input(shape=input_shape)
conv1 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(img_inputs)
conv2 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv2)
conv3 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(pool1)
conv4 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(conv3)
pool2 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv4)
conv5 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(pool2)
conv6 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(conv5)
pool3 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv6)
flatten_cnn = layers.Flatten()(pool3)
dense1 = layers.Dense(64, activation='relu')(flatten_cnn)
dense2 = layers.Dense(16, activation='relu')(dense1)
outputs = layers.Dense(1)(dense2)
Cloud2Kv3 = keras.Model(inputs=img_inputs, outputs=outputs, name="Cloud2Kv3")
Cloud2K_list.append(Cloud2Kv3)
Cloud2Kv3.summary()

# Define the CNN architecture
input_shape = (16, 16, 1)
img_inputs = keras.Input(shape=input_shape)
conv1 = layers.Conv2D(8, (5, 5), padding='same')(img_inputs)
batch1 = layers.BatchNormalization()(conv1)
act1 = layers.Activation(activation='relu')(batch1)
conv2 = layers.Conv2D(8, (5, 5), padding='same')(act1)
batch2 = layers.BatchNormalization()(conv2)
act2 = layers.Activation(activation='relu')(batch2)
pool1 = layers.MaxPooling2D((2, 2),strides=(2,2))(act2)
conv3 = layers.Conv2D(16, (1, 1), padding='same')(pool1)
batch3 = layers.BatchNormalization()(conv3)
act3 = layers.Activation(activation='relu')(batch3)
resbox1 = resblock(batch3, (5, 5), 16)
resbox2 = resblock(resbox1, (5, 5), 16)
resbox3 = resblock(resbox2, (5, 5), 16)
pool3 = layers.MaxPooling2D((2, 2),strides=(2,2))(resbox3)
conv4 = layers.Conv2D(32, (1, 1), padding='same')(pool3)
batch4 = layers.BatchNormalization()(conv4)
act4 = layers.Activation(activation='relu')(batch4)
resbox4 = resblock(act4, (5, 5), 32)
resbox5 = resblock(resbox4, (5, 5), 32)
resbox6 = resblock(resbox5, (5, 5), 32)
pool4 = layers.AveragePooling2D((4, 4))(resbox6)
flatten_cnn = layers.Flatten()(pool4)
dense1 = layers.Dense(32,activation='relu')(flatten_cnn)
dense2 = layers.Dense(8,activation='relu')(dense1)
outputs = layers.Dense(1)(dense2)
Cloud2Kv4 = keras.Model(inputs=img_inputs, outputs=outputs, name="Cloud2Kv4")
Cloud2K_list.append(Cloud2Kv4)
Cloud2Kv4.summary()

# Cloud2I Model

In [ ]:
Cloud2I_name = ['Cloud2Iv1','Cloud2Iv2','Cloud2Iv3']

In [ ]:
def resblock(x, kernelsize, filters):
    fx = layers.Conv2D(filters, kernelsize, padding='same')(x)
    act1 = layers.Activation(activation='relu')(fx)
    fx = layers.Conv2D(filters, kernelsize, padding='same')(act1)
    out = layers.Add()([x,fx])
    out = layers.ReLU()(out)
    return out

In [ ]:
Cloud2I_list = []
# define CNN model
input_shape = (16, 16, 1)
img_inputs = keras.Input(shape=input_shape)
ATT_input = keras.Input(shape=(4))
conv1 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(img_inputs)
conv2 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv2)
flatten_cnn = layers.Flatten()(pool1)
dense1 = layers.Dense(4)(ATT_input) # define MLP model
flatten_mlp = layers.Flatten()(dense1)
merged = layers.concatenate([flatten_cnn, flatten_mlp]) # combine CNN and MLP
dense3 = layers.Dense(256, activation='relu')(merged) 
dense4 = layers.Dense(32, activation='relu')(dense3)
outputs = layers.Dense(1)(dense4)
Cloud2Iv1 = keras.Model(inputs=[img_inputs, ATT_input], outputs=outputs, name="Cloud2Iv1")
Cloud2I_list.append(Cloud2Iv1)
Cloud2Iv1.summary()

# define CNN model
input_shape = (16, 16, 1)
img_inputs = keras.Input(shape=input_shape)
ATT_input = keras.Input(shape=(4))
conv1 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(img_inputs)
conv2 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv2)
conv3 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(pool1)
conv4 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(conv3)
pool2 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv4)
conv5 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(pool2)
conv6 = layers.Conv2D(16, kernel_size=(5, 5), activation='relu',padding='same')(conv5)
pool3 = layers.MaxPooling2D((2, 2),strides=(2,2))(conv6)
flatten_cnn = layers.Flatten()(pool3)
dense1 = layers.Dense(4)(ATT_input) # define MLP model
flatten_mlp = layers.Flatten()(dense1)
merged = layers.concatenate([flatten_cnn, flatten_mlp]) # combine CNN and MLP
dense3 = layers.Dense(64, activation='relu')(merged)
dense4 = layers.Dense(16, activation='relu')(dense3)
outputs = layers.Dense(1)(dense4)
Cloud2Iv2 = keras.Model(inputs=[img_inputs, ATT_input], outputs=outputs, name="Cloud2Iv2")
Cloud2I_list.append(Cloud2Iv2)
Cloud2Iv2.summary()


# define CNN model
input_shape = (16, 16, 1)
img_inputs = keras.Input(shape=input_shape)
ATT_input = keras.Input(shape=(4))
conv1 = layers.Conv2D(8, (5, 5), padding='same')(img_inputs)
batch1 = layers.BatchNormalization()(conv1)
act1 = layers.Activation(activation='relu')(batch1)
conv2 = layers.Conv2D(8, (5, 5), padding='same')(act1)
batch2 = layers.BatchNormalization()(conv2)
act2 = layers.Activation(activation='relu')(batch2)
pool1 = layers.MaxPooling2D((2, 2),strides=(2,2))(act2)
conv3 = layers.Conv2D(16, (1, 1), padding='same')(pool1)
batch3 = layers.BatchNormalization()(conv3)
act3 = layers.Activation(activation='relu')(batch3)
resbox1 = resblock(batch3, (5, 5), 16)
resbox2 = resblock(resbox1, (5, 5), 16)
resbox3 = resblock(resbox2, (5, 5), 16)
pool3 = layers.MaxPooling2D((2, 2),strides=(2,2))(resbox3)
conv4 = layers.Conv2D(32, (1, 1), padding='same')(pool3)
batch4 = layers.BatchNormalization()(conv4)
resbox4 = resblock(batch4, (5, 5), 32)
resbox5 = resblock(resbox4, (5, 5), 32)
resbox6 = resblock(resbox5, (5, 5), 32)
pool4 = layers.AveragePooling2D((4, 4))(resbox6)
flatten_cnn = layers.Flatten()(pool4)
dense1 = layers.Dense(4)(ATT_input) # define MLP model
flatten_mlp = layers.Flatten()(dense1)
merged = layers.concatenate([flatten_cnn, flatten_mlp])
dense2 = layers.Dense(32,activation='relu')(merged)
dense3 = layers.Dense(8,activation='relu')(dense2)
outputs = layers.Dense(1)(dense3)
Cloud2Iv3 = keras.Model(inputs=[img_inputs, ATT_input], outputs=outputs, name="Cloud2Iv3")
Cloud2I_list.append(Cloud2Iv3)
Cloud2Iv3.summary()


In [ ]:
#Train Cloud2K model
TargetVariable=['k']
Predictors=list(df.columns.drop(['site','k','Iclr','CI0','I','date']))
 
X_train=train_data[Predictors].values.reshape(-1,16,16,1)
y_train=train_data[TargetVariable].values
X_test = test_data[Predictors].values.reshape(-1,16,16,1)
y_test =test_data[TargetVariable].values

X_train=X_train/255
X_test=X_test/255


from sklearn.model_selection import train_test_split
#split train and validate set 
X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, shuffle = False) 

In [ ]:
# Compile and train the Cloud2K model
from keras.optimizers import RMSprop,Adam
from keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

batch_size = 32
epochs=500


for index, model in enumerate(Cloud2K_list) :
    earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 50, 
                                        restore_best_weights = True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.001)
    optimizer = keras.optimizers.Adam(learning_rate=0.002)
    
    model.compile(optimizer = optimizer, loss = 'mean_absolute_error')
    Cloud2K_name[index] = model.fit(X_train_train, y_train_train, epochs = epochs, 
                        batch_size = batch_size,validation_data = (X_valid, y_valid) ,
                        verbose = 1,callbacks = [earlystopping, reduce_lr] ) 

In [ ]:
#save Cloud2K model

name = ['Cloud2K_1','Cloud2K_2','Cloud2K_3','Cloud2K_4']
for i,save_model in enumerate(Cloud2K_list):
    save_model.save(str(name[i]+'.h5'))

In [ ]:
#save loss and val_loss in each epoch to .CSV

hist_csv_file = ['hisc2kv1.csv','hisc2kv2.csv','hisc2kv3.csv','hisc2kv4.csv']
for i,model in enumerate(Cloud2K_name):
    loss = model.history['loss']
    val_loss = model.history['val_loss']
    epochs = range(len(loss))

    valid_loss = pd.DataFrame(dict(epochs = range(len(loss)),loss = model.history['loss'],val_loss = model.history['val_loss']))
    
    with open(hist_csv_file[i], mode='w') as f:
        valid_loss.to_csv(f)

In [ ]:
#Train Cloud2I model

TargetVariable = ['I']
Predictors = list(df.columns.drop(['site','k','Iclr','CI0','I','date']))

X_train = train_data[Predictors].values.reshape(-1,16,16,1)
y_train = train_data[TargetVariable].values
X_test = test_data[Predictors].values.reshape(-1,16,16,1)
y_test = test_data[TargetVariable].values

X_train = X_train/255
X_test = X_test/255

ATT_taget = ['I']
ATT_predictors = list(ATT.columns.drop(['I','site','k']))
X_train_ATT = train_ATT[ATT_predictors].values
y_train_ATT = train_ATT[ATT_taget].values
X_test_ATT = test_ATT[ATT_predictors].values
y_test_ATT = test_ATT[ATT_taget].values

from sklearn.model_selection import train_test_split
#split train and validate set 
X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, shuffle = False) 
X_train_train_ATT, X_val_ATT, y_train_train_ATT, y_val_ATT = train_test_split(X_train_ATT, y_train_AT, test_size=0.25, shuffle = False)


In [ ]:
# Compile and train the Cloud2I model

from keras.optimizers import RMSprop,Adam
from keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

batch_size = 32
epochs=500

for index,model in enumerate(Cloud2I_list) :
    earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.001)
    optimizer = keras.optimizers.Adam(lr=0.002)
    model.compile(optimizer=optimizer, loss='mean_absolute_error')
    Cloud2I_name[index]=model.fit([X_train_train, X_train_train_ATT], y_train_train, epochs=epochs, 
                              batch_size = batch_size, validation_data=([X_valid, X_val_ATT], y_valid) ,
                              verbose=1,callbacks =[earlystopping,reduce_lr] ) 


In [ ]:
#save Cloud2I model
name = ['Cloud2I_1','Cloud2I_2','Cloud2I_3']
for i,save_model in enumerate(Cloud2I_list):
    save_model.save(str(name[i]+'.h5'))

In [ ]:
#save loss and val_loss in each epoch to .CSV

hist_csv_file = ['hisc2I_1.csv','hisc2I_2.csv','hisc2I_3.csv']
for i,model in enumerate(Cloud2I_name):
    loss = model.history['loss']
    val_loss = model.history['val_loss']
    epochs = range(len(loss))

    valid_loss = pd.DataFrame(dict(epochs = range(len(loss)),loss = model.history['loss'],val_loss = model.history['val_loss']))
    
    with open(hist_csv_file[i], mode='w') as f:
        valid_loss.to_csv(f)
        


# Model testing 

In [ ]:
file_name = os.listdir('/Users/khunnoot/Desktop/year4/senior_project/jupyter_notebook')

file_Cloud2K = ['Cloud2K_1.h5','Cloud2K_2.h5','Cloud2K_3.h5','Cloud2K_4.h5']
file_Cloud2I = ['Cloud2I_1.h5','Cloud2I_2.h5','Cloud2I_3.h5']

In [ ]:
from tensorflow.keras.models import load_model

def model(file): #load model and keep in the list 
    model = []
    for name in file:
        model.append(load_model(os.path.join(Directory,name)))
    return model
def get_name(file) : # get name of each model and add in the list
    name = []
    for i in file :
        i = i[:-3]
        name.append(i)
    return name 
def k_to_I(df,k_columns,I_columns): #change estimated k to estimated I
    col_name = []
    for i,c in enumerate(k_columns):
        name = 'I_'+c
        df=df.assign( col = lambda x: (x[c] * x['Iclr']))
        df.rename(columns={'col':name},inplace=True)
        col_name.append(name)
    for i in I_columns :
        name = 'I_' + i
        col_name.append(name)
    return df[['site','I']+col_name]
def AE(df,I_columns): #generate absolute error
    name = []
    for i,c in enumerate(I_columns):
        AE_name = 'AE_'+c[2:]
        df=df.assign( col = lambda x: abs(x[c] - x['I']))
        df.rename(columns={'col':AE_name},inplace=True)
        SE_name = 'SE_'+c[2:]
        df=df.assign( col = lambda x: (x[c] - x['I'])**2)
        df.rename(columns={'col':SE_name},inplace=True)
        name.append(AE_name)
    return df[name]

In [ ]:
model_Cloud2K = model(file_Cloud2K)
model_Cloud2I = model(file_Cloud2I)
Cloud2K = get_name(file_Cloud2K)
Cloud2I =  get_name(file_Cloud2I)

In [ ]:
# pred_model generate the model prediction when the input is 'df,ATT' = dataframe, Cloud2Kname and Cloud2Iname is 
# is list of name of each models , model_Cloud2K and model_Cloud2I is list of each models 
# if you do not have any model, you can insert only '[]'

def pred_model(df, ATT, Cloud2Kname, Cloud2Iname, model_Cloud2K, model_Cloud2I):
    #train test split
    train_data, test_data = train_test_splits(df, test_size=0.2, random_state=42, split_by_k = True)
    train_ATT, test_ATT = train_test_splits(ATT, test_size=0.2, random_state=42, split_by_k = True)

    #for map k
    TargetVariable=['k']
    Predictors=list(df.columns.drop(['site','k','Iclr','CI0','I','date']))
    X_train=train_data[Predictors].values.reshape(-1,16,16,1)
    y_train=train_data[TargetVariable].values
    X_test = test_data[Predictors].values.reshape(-1,16,16,1)
    y_test =test_data[TargetVariable].values
    X_train=X_train/255
    X_test=X_test/255

    from sklearn.model_selection import train_test_split
    #split train and validate set 
    X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, shuffle = False) 

    ## run train test split of model map k
    result=test_data[['site','Iclr','I']+TargetVariable].copy()
    for i,model in enumerate(model_Cloud2K):
        k_pred=model.predict(X_test)
        result['k_predict']=k_pred
        result=result.rename(columns={'k_predict':Cloud2Kname[i]})

    #for map I

    TargetVariable=['I']
    Predictors=list(df.columns.drop(['site','k','Iclr','CI0','I','date']))
    X_train=train_data[Predictors].values.reshape(-1,16,16,1)
    y_train=train_data[TargetVariable].values
    X_test = test_data[Predictors].values.reshape(-1,16,16,1)
    y_test =test_data[TargetVariable].values
    X_train=X_train/255
    X_test=X_test/255

    ATT_taget = ['I']
    ATT_predictors = list(ATT.columns.drop(['I','site','k','date']))
    X_train_ATT = train_ATT[ATT_predictors].values
    y_train_ATT = train_ATT[ATT_taget].values
    X_test_ATT = test_ATT[ATT_predictors].values
    y_test_ATT =test_ATT[ATT_taget].values

    from sklearn.model_selection import train_test_split
    #split train and validate set 
    X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, shuffle = False) 
    X_train_train_ATT, X_val_ATT, y_train_train_ATT, y_val_ATT = train_test_split(X_train_ATT, y_train_ATT, test_size=0.25, shuffle = False)

    ## run train test split of model map I
    for i,model in enumerate(model_Cloud2I):
        I_pred=model.predict([X_test,X_test_ATT])
        result['I_predict']=I_pred
        result=result.rename(columns={'I_predict':'I_'+Cloud2Iname[i]})
    
    return result

In [ ]:
output = pred_model(df,ATT,Cloud2K, Cloud2I, model_Cloud2K, model_Cloud2I)

In [ ]:
Ihat = k_to_I(output, Cloud2K, Cloud2I)
Ihat

In [ ]:
#Ihat.to_csv('Ihat_1layer.csv')

In [ ]:
#MAE in each model
I_col = Ihat.drop(columns=['site','I'])
err_AE = AE(Ihat,I_col)
err_AE.mean()